In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Gemini 2.5 Pro to create MCP Server



<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  
  
  
  <td style="text-align: center">
    <a href="https://github.com/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview
The Model Context Protocol (MCP) is an open standard that streamlines the integration of AI assistants with external data sources, tools, and systems. [MCP standardizes how applications provide context to LLMs](https://modelcontextprotocol.io/introduction). MCP establishes the essential standardized interface allowing AI models to connect directly with diverse external systems and services.

Developers have the option to use third-party MCP servers or create custom ones when building applications. 


This notebook demonstrates how to use Gemini 2.5 pro to create MCP servers.

### Options
1. Use Gemini 2.5 Pro via Vertex AI  
- Set up a Google cloud project 
2. Use Gemini 2.5 Pro  via Google API
- Get a Google API key
- Save the Google API as 'google_api_key' in your google cloud secret manager (optional)

## Get started

### Install Google Gen AI SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp geopy black google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Option 1 Use Vertex AI 

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [21]:
# Use the environment variable if the user doesn't provide Project ID.
import os

from google import genai

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [15]:
from google.cloud import secretmanager
from google import genai
import datetime

from google.genai.types import (
    GenerateContentConfig,
)

from typing import List, Dict, Any
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio
from google.genai import types
from typing import List

import sys
import os

sys.path.append(os.path.abspath("/util"))
from util.util import access_secret_version, get_url_content, format_python

### Option 2 Use Google API key 

In [ ]:
# You need to create a 'google_api_key' and save it in secret manager. If using API key, please uncomment the below code to run. 
# api_key = access_secret_version(
#     PROJECT_ID, secret_id="google_api_key", version_id="latest"
# )
# Alternatively, get google API key from environment if you've set it up.
# api_key=os.getenv("GEMINI_API_KEY")
# client = genai.Client(api_key=api_key)

In [4]:
# TODO(developer): Update below line
MODEL25_ID = (
    "gemini-2.5-pro-exp-03-25"  # This model is only for MCP server code generation.
)

### Get system instruction context info

In [5]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"

In [9]:
reference_content = get_url_content(url)

Successfully fetched content


### Set up system instruction

In [10]:
from pydantic import BaseModel


class response_schema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""

In [11]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL25_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=response_schema,
        ),
    )

    return response.text

### Gemini Agent

Within an MCP client session, this agent loop runs a multi-turn conversation loop with a Gemini model, handling tool calls via MCP server.

This function orchestrates the interaction between a user prompt, a Gemini model capable of function calling, and a session object that provides and executes tools. It handles the cycle of:
-  Gemini gets tool information from MCP client session
-  Sending the user prompt (and conversation history) to the model.
-  If the model requests tool calls, Gemini makes initial function calls to get structured data as per schema, and 
-  Sending the tool execution results back to the model.
-  Repeating until the model provides a text response or the maximum number of tool execution turns is reached.
-  Gemini generates final response based on tool responses and original query.
  
MCP integration with Gemini

<img src="asset/mcp_tool_call.png" alt="MCP with Gemini" height="700">

In [12]:
# --- Configuration ---
# Consider using a more recent/recommended model if available and suitable
MODEL_ID = "gemini-2.5-pro-exp-03-25"  # You can use a different model for the agent
DEFAULT_MAX_TOOL_TURNS = 5  # Maximum consecutive turns for tool execution
DEFAULT_INITIAL_TEMPERATURE = (
    0.0  # Temperature for the first LLM call (more deterministic)
)
DEFAULT_TOOL_CALL_TEMPERATURE = (
    1.0  # Temperature for LLM calls after tool use (potentially more creative)
)

# Make tool calls via MCP Server


async def _execute_tool_calls(
    function_calls: List[types.FunctionCall], session: ClientSession
) -> List[types.Part]:
    """
    Executes a list of function calls requested by the Gemini model via the session.

    Args:
        function_calls: A list of FunctionCall objects from the model's response.
        session: The session object capable of executing tools via `call_tool`.

    Returns:
        A list of Part objects, each containing a FunctionResponse corresponding
        to the execution result of a requested tool call.
    """
    tool_response_parts: List[types.Part] = []
    print(f"--- Executing {len(function_calls)} tool call(s) ---")

    for func_call in function_calls:
        tool_name = func_call.name
        # Ensure args is a dictionary, even if missing or not a dict type
        args = func_call.args if isinstance(func_call.args, dict) else {}
        print(f"  Attempting to call session tool: '{tool_name}' with args: {args}")

        tool_result_payload: Dict[str, Any]
        try:
            # Execute the tool using the provided session object
            # Assumes session.call_tool returns an object with attributes
            # like `isError` (bool) and `content` (list of Part-like objects).
            tool_result = await session.call_tool(tool_name, args)
            print(f"  Session tool '{tool_name}' execution finished.")

            # Extract result or error message from the tool result object
            result_text = ""
            # Check structure carefully based on actual `session.call_tool` return type
            if (
                hasattr(tool_result, "content")
                and tool_result.content
                and hasattr(tool_result.content[0], "text")
            ):
                result_text = tool_result.content[0].text or ""

            if hasattr(tool_result, "isError") and tool_result.isError:
                error_message = (
                    result_text
                    or f"Tool '{tool_name}' failed without specific error message."
                )
                print(f"  Tool '{tool_name}' reported an error: {error_message}")
                tool_result_payload = {"error": error_message}
            else:
                print(
                    f"  Tool '{tool_name}' succeeded. Result snippet: {result_text[:150]}..."
                )  # Log snippet
                tool_result_payload = {"result": result_text}

        except Exception as e:
            # Catch exceptions during the tool call itself
            error_message = f"Tool execution framework failed: {type(e).__name__}: {e}"
            print(f"  Error executing tool '{tool_name}': {error_message}")
            tool_result_payload = {"error": error_message}

        # Create a FunctionResponse Part to send back to the model
        tool_response_parts.append(
            types.Part.from_function_response(
                name=tool_name, response=tool_result_payload
            )
        )
    print(f"--- Finished executing tool call(s) ---")
    return tool_response_parts


async def run_agent_loop(
    prompt: str,
    client: genai.Client,
    session: ClientSession,
    model_id: str = MODEL_ID,
    max_tool_turns: int = DEFAULT_MAX_TOOL_TURNS,
    initial_temperature: float = DEFAULT_INITIAL_TEMPERATURE,
    tool_call_temperature: float = DEFAULT_TOOL_CALL_TEMPERATURE,
) -> types.GenerateContentResponse:
    """
    Runs a multi-turn conversation loop with a Gemini model, handling tool calls.

    This function orchestrates the interaction between a user prompt, a Gemini
    model capable of function calling, and a session object that provides
    and executes tools. It handles the cycle of:
    1. Sending the user prompt (and conversation history) to the model.
    2. If the model requests tool calls, executing them via the `session`.
    3. Sending the tool execution results back to the model.
    4. Repeating until the model provides a text response or the maximum
       number of tool execution turns is reached.

    Args:
        prompt: The initial user prompt to start the conversation.
        client: An initialized Gemini GenerativeModel client object

        session: An active session object responsible for listing available tools
                 via `list_tools()` and executing them via `call_tool(tool_name, args)`.
                 It's also expected to have an `initialize()` method.
        model_id: The identifier of the Gemini model to use (e.g., "gemini-1.5-pro-latest").
        max_tool_turns: The maximum number of consecutive turns dedicated to tool calls
                        before forcing a final response or exiting.
        initial_temperature: The temperature setting for the first model call.
        tool_call_temperature: The temperature setting for subsequent model calls
                               that occur after tool execution.

    Returns:
        The final Response from the Gemini model after the
        conversation loop concludes (either with a text response or after
        reaching the max tool turns).

    Raises:
        ValueError: If the session object does not provide any tools.
        Exception: Can potentially raise exceptions from the underlying API calls
                   or session tool execution if not caught internally by `_execute_tool_calls`.
    """
    print(
        f"Starting agent loop with model '{model_id}' and prompt: '{prompt[:100]}...'"
    )

    # Initialize conversation history with the user's prompt
    contents: List[types.Content] = [
        types.Content(role="user", parts=[types.Part(text=prompt)])
    ]

    # Ensure the session is ready (if needed)
    if hasattr(session, "initialize") and callable(session.initialize):
        print("Initializing session...")
        await session.initialize()
    else:
        print("Session object does not have an initialize() method, proceeding anyway.")

    # --- 1. Discover Tools from Session ---
    print("Listing tools from session...")
    # Assumes session.list_tools() returns an object with a 'tools' attribute (list)
    # Each item in the list should have 'name', 'description', and 'inputSchema' attributes.
    session_tool_list = await session.list_tools()

    if not session_tool_list or not session_tool_list.tools:
        raise ValueError("No tools provided by the session. Agent loop cannot proceed.")

    # Convert session tools to the format required by the Gemini API
    gemini_tool_config = types.Tool(
        function_declarations=[
            types.FunctionDeclaration(
                name=tool.name,
                description=tool.description,
                parameters=tool.inputSchema,  # Assumes inputSchema is compatible
            )
            for tool in session_tool_list.tools
        ]
    )
    print(
        f"Configured Gemini with {len(gemini_tool_config.function_declarations)} tool(s)."
    )

    # --- 2. Initial Model Call ---
    print("Making initial call to Gemini model...")
    current_temperature = initial_temperature
    response = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=contents,  # Send updated history
        config=types.GenerateContentConfig(
            temperature=1.0,
            tools=[gemini_tool_config],
        ),  # Keep sending same config
    )
    print("Initial response received.")

    # Append the model's first response (potentially including function calls) to history
    # Need to handle potential lack of candidates or content
    if not response.candidates:
        print("Warning: Initial model response has no candidates.")
        # Decide how to handle this - raise error or return the empty response?
        return response
    contents.append(response.candidates[0].content)

    # --- 3. Tool Calling Loop ---
    turn_count = 0
    # Check specifically for FunctionCall objects in the latest response part
    latest_content = response.candidates[0].content
    has_function_calls = any(part.function_call for part in latest_content.parts)

    while has_function_calls and turn_count < max_tool_turns:
        turn_count += 1
        print(f"\n--- Tool Turn {turn_count}/{max_tool_turns} ---")

        # --- 3.1 Execute Pending Function Calls ---
        function_calls_to_execute = [
            part.function_call for part in latest_content.parts if part.function_call
        ]
        tool_response_parts = await _execute_tool_calls(
            function_calls_to_execute, session
        )

        # --- 3.2 Add Tool Responses to History ---
        # Send back the results for *all* function calls from the previous turn
        contents.append(
            types.Content(role="function", parts=tool_response_parts)
        )  # Use "function" role
        print(f"Added {len(tool_response_parts)} tool response part(s) to history.")

        # --- 3.3 Make Subsequent Model Call with Tool Responses ---
        print("Making subsequent API call to Gemini with tool responses...")
        current_temperature = tool_call_temperature  # Use different temp for follow-up
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[gemini_tool_config],
            ),
        )
        print("Subsequent response received.")

        # --- 3.4 Append latest model response and check for more calls ---
        if not response.candidates:
            print("Warning: Subsequent model response has no candidates.")
            break  # Exit loop if no candidates are returned
        latest_content = response.candidates[0].content
        contents.append(latest_content)
        has_function_calls = any(part.function_call for part in latest_content.parts)
        if not has_function_calls:
            print(
                "Model response contains text, no further tool calls requested this turn."
            )

    # --- 4. Loop Termination Check ---
    if turn_count >= max_tool_turns and has_function_calls:
        print(
            f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop even though function calls might be pending."
        )
    elif not has_function_calls:
        print("Tool calling loop finished naturally (model provided text response).")

    # --- 5. Return Final Response ---
    print("Agent loop finished. Returning final response.")
    return response

## Example 1:  Create MCP Server for Google Cloud BigQuery

In [33]:
prompt = """
  Please create an MCP server code for google cloud big query. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google cloud project id and location will be provided for use. please use project id to access big query client.
  
  Return Python code within a JSON object formatted exactly as: {'python_code':    'your_generated_code', 'description':'your description'}
"""

In [12]:
response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

In [20]:
format_python(python_code, "server/bq.py")

Successfully formatted the code and saved it to 'server/bq.py'


In [13]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [17]:
async def run():
    async with stdio_client(bq_server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res


res = await run()
print(res.text)

Running agent loop with prompt: Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'...'
Initializing session...
Listing tools from session...
Configured Gemini with 2 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'list_tables' with args: {'project_id': 'dw-genai-dev'}
  Session tool 'list_tables' execution finished.
  Tool 'list_tables' succeeded. Result snippet: demo_dataset1.item_table
demo_dataset1.user_table
demo_dataset2.item_table
demo_dataset2.user_table...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested 

### Example 2:  Create MCP server for Medlineplus website
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [15]:
med_url = "https://medlineplus.gov/about/developers/webservices/"

In [16]:
med_api_details = get_url_content(med_url)

Successfully fetched content


In [17]:
prompt = f"""
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term
  
  Here's the API details:
  {med_api_details}
"""

response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

format_python(python_code, "server/med.py")

Successfully formatted the code and saved it to 'server/med.py'


In [18]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [19]:
async def run():
    async with stdio_client(med_server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "Please explain flu in detail."
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res


res = await run()
print(res.text)

Running agent loop with prompt: Please explain flu in detail.
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please explain flu in detail....'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_medical_term' with args: {'medical_term': 'flu'}
  Session tool 'get_medical_term' execution finished.
  Tool 'get_medical_term' succeeded. Result snippet: Term: <span class="qt0">Flu</span>
URL: https://medlineplus.gov/flu.html

Summary:
What is the <span class="qt0">flu</span>?<p>The <span class="qt0">f...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished n

### Example 3: Create MCP Server for NIH

In [20]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"

In [21]:
nih_api_details = get_url_content(nih_url)

Successfully fetched content


In [22]:
prompt = f"""
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results. 
  
  Here's the API details:
   {nih_api_details}
"""

response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

format_python(python_code, "server/nih.py")

Successfully formatted the code and saved it to 'server/nih.py'


In [20]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)


async def run():
    async with stdio_client(nih_server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "Please tell me ICD_10 code for Tuberculosis"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res


res = await run()
print(res.text)

Running agent loop with prompt: Please tell me ICD_10 code for Tuberculosis
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please tell me ICD_10 code for Tuberculosis...'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_icd_10_code' with args: {'query': 'Tuberculosis'}
  Session tool 'get_icd_10_code' execution finished.
  Tool 'get_icd_10_code' succeeded. Result snippet: Code: A18.39, Name: Retroperitoneal tuberculosis
Code: P37.0, Name: Congenital tuberculosis
Code: Z22.7, Name: Latent tuberculosis
Code: A15.0, Name: ...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
T